In [2]:
from sqlalchemy import create_engine #, MetaData, Table,select, and_
from sqlalchemy.engine import URL, result
import pandas as pd
import numpy as np
from sqlalchemy.orm import Session, sessionmaker

In [3]:

connection_url = URL.create(
  'mssql+pyodbc',
  username='sa',
  password='spts@3311',
  host='10.0.0.9',
  port=1433,
  database='ShahkamSpts',
  query={
    'driver': 'ODBC Driver 17 for SQL Server',
  },
)
engine = create_engine(connection_url)
session = Session(engine, future=True)


In [49]:
worker = session.execute('select top 1000 CreatedAt, WorkerID, TotalPiecesToday, LoginTimeToday from essentials.worker').all()
worker_df = pd.DataFrame(worker, columns=['CreatedAt', 'WorkerID', 'TotalPiecesToday', 'LoginTimeToday'])
# print(np.shape(worker_df))
# print(worker_df.head())


In [50]:
# #worker scan data
worker_scan = session.execute('select top 1000 CreatedAt, WorkerID, MachineID from Data.WorkerScan').all()
workerscan_df = pd.DataFrame(worker_scan, columns=['CreatedAt', 'WorkerID', 'MachineID'])
# print(np.shape(workerscan_df))
# print(workerscan_df.head())

In [51]:
# TODO: staytime calculation
bundle_scan = session.execute('select top 1000  CreatedAt, WorkerID, BundleScanID, BundleQuantity, PieceRate from data.BundleScan').all()
bundlescan_df = pd.DataFrame(bundle_scan, columns=['CreatedAt', 'WorkerID', 'BundleScanID', 'BundleQuantity', 'PieceRate']).fillna(0)
# print(bundlescan_df)

logout = bundlescan_df.groupby('WorkerID').tail(1)
login = worker_df.groupby("WorkerID").head(1)

# logout = pd.DataFrame(logout).sort_index("CreatedAt", ascending=True)
# login = pd.DataFrame(login).sort_index("CreatedAt", ascending=True)

logout.sort_values("WorkerID", ascending=True)[:10]

,CreatedAt,WorkerID,BundleScanID,BundleQuantity,PieceRate
931,2021-01-14 14:58:04.530,11,1019,20,60.00
893,2021-01-14 14:46:25.320,12,981,14,60.00
870,2021-01-14 14:42:41.867,13,957,20,60.00
796,2021-01-14 14:16:44.783,14,883,16,60.00
577,2021-01-14 12:31:09.660,18,662,14,11.32
814,2021-01-14 14:28:01.390,21,901,18,60.00
846,2021-01-14 14:34:17.167,43,933,20,60.00
504,2021-01-13 16:48:21.737,50,589,24,12.48
915,2021-01-14 14:50:27.047,54,1003,14,60.00
530,2021-01-13 16:55:32.773,57,615,24,15.60


In [52]:
login.sort_values("WorkerID", ascending=True)[:10]

,CreatedAt,WorkerID,TotalPiecesToday,LoginTimeToday
0,2021-01-07 18:13:36.040,10,18,07:19
1,2021-01-07 18:13:36.040,11,0,None
2,2021-01-07 18:13:36.040,12,46,08:16
3,2021-01-07 18:13:36.040,13,258,08:09
4,2021-01-07 18:13:36.040,14,0,None
5,2021-01-07 18:13:36.040,15,638,08:08
6,2021-01-07 18:13:36.040,16,177,08:59
7,2021-01-07 18:13:36.040,17,365,08:56
8,2021-01-07 18:13:36.040,18,200,08:15
9,2021-01-07 18:13:36.040,19,0,None


In [53]:
staytime = logout.set_index('WorkerID').subtract(login.set_index('WorkerID')).reset_index()
staytime.head(20)

,WorkerID,BundleQuantity,BundleScanID,CreatedAt,LoginTimeToday,PieceRate,TotalPiecesToday
0,10,NaN,NaN,NaT,NaN,NaN,NaN
1,11,NaN,NaN,6 days 20:44:28.490000,NaN,NaN,NaN
2,12,NaN,NaN,6 days 20:32:49.280000,NaN,NaN,NaN
3,13,NaN,NaN,6 days 20:29:05.827000,NaN,NaN,NaN
4,14,NaN,NaN,6 days 20:03:08.743000,NaN,NaN,NaN
5,15,NaN,NaN,NaT,NaN,NaN,NaN
6,16,NaN,NaN,NaT,NaN,NaN,NaN
7,17,NaN,NaN,NaT,NaN,NaN,NaN
8,18,NaN,NaN,6 days 18:17:33.620000,NaN,NaN,NaN
9,19,NaN,NaN,NaT,NaN,NaN,NaN


In [78]:
# TODO: wage calculation

# wage = bundlescan_df['BundleQuantity'] * bundlescan_df["PieceRate"]
# wage

bundlescan_df['wage'] = bundlescan_df['BundleQuantity'].mul(bundlescan_df['PieceRate'])
bundlescan_df

,CreatedAt,WorkerID,BundleScanID,BundleQuantity,PieceRate,wage
0,2021-01-13 09:41:46.323,271,24,8,0.00,0.00
1,2021-01-13 10:26:56.540,271,64,20,0.00,0.00
2,2021-01-13 11:35:05.920,271,87,20,0.00,0.00
3,2021-01-13 11:35:12.460,271,88,18,0.00,0.00
4,2021-01-13 11:35:21.200,271,89,18,0.00,0.00
...,...,...,...,...,...,...
995,2021-01-14 15:02:47.570,239,1084,16,11.32,181.12
996,2021-01-14 15:02:47.840,3884,1085,14,10.32,144.48
997,2021-01-14 15:02:49.660,3884,1086,6,10.32,61.92
998,2021-01-14 15:02:50.843,239,1087,18,11.32,203.76


In [84]:
bundlescan_df.groupby([pd.Grouper(key='CreatedAt', axis=0, 
                      freq='1D', sort=True),"WorkerID"]).sum('wage')

# bundlescan_df.groupby("WorkerID").sum('wage')

BundleScanID  BundleQuantity  PieceRate      wage
CreatedAt  WorkerID                                                   
2021-01-13 21                2968              68     300.00   4080.00
           50                2345              60      49.92    748.80
           57                1229              34      31.20    530.40
           60                4199              66      80.08    755.04
           62                1770              46      56.16    861.12
           75                 585              28      18.72    524.16
           76                1167              32      37.44    599.04
           103               9175             248     232.00   3783.84
           260                582              20      12.48    249.60
           271              52205            3832       0.00      0.00
           272             112632            3802       0.00      0.00
2021-01-14 11               16184             266     960.00  15960.00
           12               22292             396    1380.00  23760.00
           13               22692             418    1440.00  25080.00
           14               14898             298    1140.00  17880.00
           18                6575             140     113.20   1584.80
           21               21988             418    1620.00  25080.00
           43               17833             364    1320.00  21840.00
           54               21835             318    1320.00  19080.00
           60               17309             338    1140.00  20280.00
           222               5166              94      82.56    970.08
           239              13945             228     147.16   2580.96
           270               1079               8      12.48     99.84
           3884             26636             276     258.00   2848.32
           4246            115722            2200    1822.08  27456.00
           9520             42033            1006     748.80  12554.88
           14892            29986             392    1740.00  23520.00

In [86]:
# import seaborn as sns
# sns.pairplot(bundlescan_df)